In [1]:
from config_figures import *

In [41]:
splot_data.size().getInfo()

308660

In [42]:
splot_data.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [3.6539300944272632, 44.00009938105824]},
 'id': '0000000000000003ccd3',
 'properties': {'Location_uncertainty': None,
  'PlotObservationID': 482319,
  'Species': 'Alnus glutinosa'}}

In [43]:
print('sPlot data:')
print(f"{splot_data.size().getInfo()} features in feature collection")
print(f"{splot_data.distinct('Species').size().getInfo()} species")
print(f"{splot_data.distinct('.geo').size().getInfo()} distinct locations")

sPlot data:
308660 features in feature collection
5939 species
47479 distinct locations


In [44]:
sdm_species = sdms.aggregate_array('system:index').getInfo()
print(len(sdm_species), 'species: ', sdm_species[0:10])

10590 species:  ['Abarema_barbouriana', 'Abarema_brachystachya', 'Abarema_cochleata', 'Abarema_filamentosa', 'Abarema_jupunba', 'Abarema_laeta', 'Abarema_langsdorffii', 'Abatia_parviflora', 'Abies_alba', 'Abies_amabilis']


In [45]:
# splot_data = splot_data.filter(ee.Filter.inList('Species', ee.List([s.replace('_', ' ') for s in sdm_species])))
# print('sPlot data after filtering for species in SDMs:')
# print(f"{splot_data.size().getInfo()} features in feature collection")
# print(f"{splot_data.distinct('Species').size().getInfo()} species")
# print(f"{splot_data.distinct('.geo').size().getInfo()} distinct locations")

In [46]:
# splot_data = splot_data.distinct(['.geo', 'Species'])
# print('sPlot data after merging observations at same location:')
# print(f"{splot_data.size().getInfo()} features in feature collection")
# print(f"{splot_data.distinct('Species').size().getInfo()} species")
# print(f"{splot_data.distinct('.geo').size().getInfo()} distinct locations")

In [47]:
splot_unique_locations = splot_data.distinct('.geo')
print(f"Number of unique locations: {splot_unique_locations.size().getInfo()}")

Number of unique locations: 47479


In [102]:
splot_species = splot_data.distinct('Species').aggregate_array('Species').map(lambda x: ee.String(x).replace(' ','_')).getInfo()
splot_species[0:10]

['Alnus_glutinosa',
 'Corylus_avellana',
 'Fraxinus_angustifolia',
 'Populus_nigra',
 'Salix_eleagnos',
 'Salix_purpurea',
 'Buxus_sempervirens',
 'Castanea_sativa',
 'Quercus_pubescens',
 'Betula_pendula']

In [111]:
ee.List(sdm_species).filter(ee.Filter.inList('item', ee.List(splot_species))).size().getInfo()

5939

In [103]:
set(sdm_species).difference(set(splot_species))

{'Claoxylon_hillii',
 'Chamaecrista_eitenorum',
 'Lepidothamnus_intermedius',
 'Terminalia_fagifolia',
 'Diospyros_pubescens',
 'Clusia_renggerioides',
 'Syzygium_hemisphericum',
 'Croton_sylvaticus',
 'Cordia_glazioviana',
 'Talisia_cerasina',
 'Polyalthia_borneensis',
 'Annona_leptopetala',
 'Podocarpus_totara',
 'Helicteres_baruensis',
 'Tilia_japonica',
 'Inga_goldmanii',
 'Chionanthus_domingensis',
 'Helietta_parvifolia',
 'Salix_eriocephala',
 'Conandrium_polyanthum',
 'Cojoba_sophorocarpa',
 'Erythroxylum_ferrugineum',
 'Alnus_nepalensis',
 'Salvertia_convallariodora',
 'Campylospermum_squamosum',
 'Irvingia_smithii',
 'Bursera_glabrifolia',
 'Mimosa_benthamii',
 'Senna_pallida',
 'Solanum_jamaicense',
 'Abies_guatemalensis',
 'Ardisia_pellucida',
 'Ocotea_cymosa',
 'Dysoxylum_spectabile',
 'Gossypium_aridum',
 'Eucryphia_milliganii',
 'Ateleia_pterocarpa',
 'Dracaena_aletriformis',
 'Eucalyptus_neglecta',
 'Acacia_mimula',
 'Gymnopodium_floribundum',
 'Astrocaryum_urostachys',


In [104]:
species = 'Clusia_renggerioides'#sdm_species[100]
species

'Clusia_renggerioides'

In [105]:
sdm = sdms.filter(ee.Filter.eq('system:index', species)).first().select('covariates_1981_2010')
sdm_bbox = sdm_bboxes.filter(ee.Filter.eq('species', species)).first().geometry()
species_splot = splot_data.filterBounds(sdm_bbox).distinct('.geo').map(lambda f: f.set('splot_presence', 0))
print(species_splot.size().getInfo())
print(species_splot.aggregate_histogram('splot_presence').getInfo())

1296
{'0': 1296}


In [106]:
presences = splot_data.filter(ee.Filter.eq('Species', species.replace('_',' ')))
print(presences.size().getInfo())

0


In [107]:
species_splot_pa = species_splot.filter(ee.Filter.bounds(presences)).map(lambda f: f.set('splot_presence', 1)).merge(species_splot.filter(ee.Filter.bounds(presences).Not()))
print(species_splot_pa.size().getInfo())
print(species_splot_pa.aggregate_histogram('splot_presence').getInfo())

1296
{'0': 1296}


In [97]:
ee.Number(species_splot_pa.aggregate_histogram('splot_presence').values().reduce(ee.Reducer.sum())).eq(species_splot.size()).getInfo()

1

In [98]:
species_splot_pa.filter(ee.Filter.eq('splot_presence', 1)).size().eq(presences.size()).getInfo()

1

In [108]:
species_sdm_splot_pa = sdm.reduceRegions(collection = species_splot_pa, reducer = ee.Reducer.first(), scale = scale_to_use).filter(ee.Filter.notNull(['first'])).map(
    lambda f: f.select(['first', 'splot_presence'],['sdm_presence', 'splot_presence'])
)
print(species_sdm_splot_pa.size().getInfo())
print(species_sdm_splot_pa.aggregate_histogram('sdm_presence').getInfo())
print(species_sdm_splot_pa.aggregate_histogram('splot_presence').getInfo())

777
{'0': 749, '1': 28}
{'0': 777}


In [109]:
tp = species_sdm_splot_pa.filter(ee.Filter.And(ee.Filter.eq('sdm_presence', 1), ee.Filter.eq('splot_presence', 1))).size()
fp = species_sdm_splot_pa.filter(ee.Filter.And(ee.Filter.eq('sdm_presence', 1), ee.Filter.eq('splot_presence', 0))).size()
tn = species_sdm_splot_pa.filter(ee.Filter.And(ee.Filter.eq('sdm_presence', 0), ee.Filter.eq('splot_presence', 0))).size()
fn = species_sdm_splot_pa.filter(ee.Filter.And(ee.Filter.eq('sdm_presence', 0), ee.Filter.eq('splot_presence', 1))).size()
tss = (tp.divide(tp.add(fn))).add(tn.divide(fp.add(tn))).subtract(1)
print('tp: ', tp.getInfo(), ' | fp: ', fp.getInfo())
print('fn: ', fn.getInfo(), ' | tn: ', tn.getInfo())
print(tss.getInfo())

tp:  0  | fp:  28
fn:  0  | tn:  749
-0.036036036036036


In [ ]:
ee.Feature(None, {'species': species, 'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn})